Sort people into leagues for curling
KL 8/21/2016

#This version starts with a random league and puts a random subset of people into each league

In [180]:
if True:
    %reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [181]:
import pandas as pd
import numpy as np
import random
import math
import csv
import os.path

#used this to step into the function and debug it, also need line with Tracer()() 
from IPython.core.debugger import Tracer 

Define some functions up top

In [182]:
def addOne(leagueDict,oneLeague,oneName):
    if len(leagueDict[oneLeague])==0: #nothing there yet
        leagueDict[oneLeague] = oneName
    else:     #need to append
        leagueDict[oneLeague] = leagueDict[oneLeague].append(oneName)
    return leagueDict

In [183]:
def addManager(leagueDictionary,r,manager): #here r is the index in...
    useRow = r.index[0]
    oneName = regSorted.loc[useRow,['lastname', 'firstname']] 
    oneName = oneName.to_frame().transpose() #need this format to make nice output
    leagueDictionary[manager] = oneName
    regSorted.loc[useRow,manager]=np.nan
    regTracking.loc[useRow,manager]='yes'
    return leagueDictionary

Read in the data

In [184]:
#goal will be to use the version that comes out of the website
fName = 'TestPriority4.csv'
regInfo=pd.read_csv(fName)

#first, how many random numbers so I need?
nPeople = len(regInfo.index)
regInfo['randomNumCol'] = np.random.choice(range(nPeople), nPeople,replace = False)
##so now, moving back to the list of people, sort the list by the random number
#...still need to randomize people bc might have case where I can't grant first choice
regSorted = regInfo.sort_values(by = 'randomNumCol', ascending=True)
regSorted = regSorted.set_index(['randomNumCol'])

#copy so I can have an output tracking what was done
regTracking = regSorted.copy(deep=True)

setup the places to store the output

In [185]:
#need the list of open leagues and randomize the order of leagues
openLeagues = list(regInfo)[5:12]
openLeagues.remove('League_4_Men')
randLeagues = list(openLeagues) #make a copy of the list
random.shuffle(randLeagues) #does this in place, so don't make a new variable
randLeagues

['League_2_Monday_Open',
 'League_1_Sunday_Open',
 'League_6_Thursday_Open',
 'League_3_Monday_Night_Open',
 'League_5_Wednesday_Open',
 'League_Tuesday_Open']

In [186]:
#setup the dictionary to hold the people information (start with openLeagues only)
forDictionary = openLeagues
#forDictionary.extend(otherLeagues)
leagueDict_open = {}
for name in forDictionary:
    leagueDict_open[name] = pd.DataFrame()

In [187]:
#put the league managers into their league and set their the choice for that league to NaN
manager = 'League_1_Sunday_Open'
r = regSorted.loc[(regSorted['lastname']=='Fragaszy') & (regSorted['firstname']=='Bill')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_2_Monday_Open'
r = regSorted.loc[(regSorted['lastname']=='Brown') & (regSorted['firstname']=='Peter')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_3_Monday_Night_Open'
r = regSorted.loc[(regSorted['lastname']=='DiMassa') & (regSorted['firstname']=='Diane')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_Tuesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Thomson') & (regSorted['firstname']=='Kenny')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_5_Wednesday_Open'
r = regSorted.loc[(regSorted['lastname']=='Murphy') & (regSorted['firstname']=='Katie')]
leagueDict_open = addManager(leagueDict_open,r,manager)

manager = 'League_6_Thursday_Open'
r = regSorted.loc[(regSorted['lastname']=='Gallagher') & (regSorted['firstname']=='Bill')]
leagueDict_open = addManager(leagueDict_open,r,manager)


In [188]:
#set the limit for each league:
sevenTeams = 5 #change to 28 (remember league manager already in here)
fourteenTeams = 56

d = {'League_1_Sunday_Open': sevenTeams,
     'League_2_Monday_Open': sevenTeams,
     'League_3_Monday_Night_Open': fourteenTeams,
     'League_Tuesday_Open': sevenTeams,
     'League_5_Wednesday_Open': sevenTeams,
     'League_6_Thursday_Open': sevenTeams,}

nLimit = pd.DataFrame({'nLimit' : d})
del d

In [189]:
issueNames = ['notInFirst','notInSecond','notInThird','noSecondChoice','noThirdChoice']

#setup the dictionary to hold the issues
trackIssues = {}
for name in issueNames:
    trackIssues[name] = pd.DataFrame()

In [190]:
pd.options.mode.chained_assignment = None  # default='warn'

In [191]:
for oneLeague in randLeagues:    
    #how many spaces are left in this league?
    nRemaining = nLimit.loc[oneLeague][0] - len(leagueDict_open[oneLeague])
    if nRemaining > 0:      
        #find list of people interested in that league
        subset = regSorted.loc[regSorted[oneLeague]==1]
        subset.reset_index(inplace = True)
        nPeople = len(subset.index)
        subset.loc[:,('subsetRandomNumCol')] = np.random.choice(range(nPeople),nPeople, replace = False)

        for idx in range(0,nPeople): #remember nPeople in this version is only subset for a league
            oneName = subset.loc[idx,['lastname', 'firstname']] 
            oneName = oneName.to_frame().transpose() #need this format to make nice output

            #does this person want another league?
            rt = regTracking.loc[idx,openLeagues]
            cHave = len(rt[rt=='yes'])
            del rt
            
            if cHave < regTracking.loc[idx,'numberLeagues']: #person wants more 
                if subset.loc[idx,'subsetRandomNumCol'] < (nRemaining): #random # < number of places remaining
                    leagueDict_open = addOne(leagueDict_open,oneLeague,oneName)                
                    regSorted.loc[idx,oneLeague] = np.nan
                    regTracking.loc[idx,oneLeague] = 'yes'

                else:
                    #no space in first choice league:
                    trackIssues = addOne(trackIssues,'notInFirst',oneName)
                    #set their first choice to 'no' and move onto second choices
                    newI = subset.loc[idx,'randomNumCol'] ###remember: this is the index into regSorted and regTracking
                    regTracking.loc[newI,oneLeague] = 'no'
                    regSorted.loc[newI,oneLeague] = np.nan

                    #use the idx into subset to go back into regTracking
                    newPriority = regSorted.loc[newI,openLeagues].dropna().min()

                    if math.isnan(newPriority) == False: #have another league listed
                        league2 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                        #first need to check if the league has space
                        if len(leagueDict_open[league2]) < (nLimit.loc[league2,'nLimit']):
                            leagueDict_open = addOne(leagueDict_open,league2,oneName)
                            regTracking.loc[newI,league2] = 'yes'
                            regSorted.loc[newI,league2] = np.nan
                        else:
                            #no space in second choice league
                            trackIssues = addOne(trackIssues,'notInSecond',oneName)
                            #set their first choice to 'no' and move onto second choices
                            regTracking.loc[newI,league2] = 'no' 
                            regSorted.loc[newI,league2] = np.nan

                            #use the idx into subset to go back into regTracking
                            priority3 = regSorted.loc[newI,openLeagues].dropna().min()

                            if math.isnan(priority3) == False: #have another league listed
                                league3 = regSorted.loc[newI,openLeagues].dropna().idxmin() #returns name of league
                                #first need to check if the league has space
                                if len(leagueDict_open[league3]) < (nLimit.loc[league3,'nLimit']):
                                    leagueDict_open = addOne(leagueDict_open,league3,oneName)
                                    regTracking.loc[newI,league3] = 'yes'
                                    regSorted.loc[newI,league3] = np.nan
                                else:
                                    print oneName.values.ravel()[1],oneName.values.ravel()[0],': no space in third choice league'
                                    trackIssues = addOne(trackIssues,'notInThird',oneName)
                                    regTracking.loc[newI,league3] = 'no'
                            else: 
                                #no other league listed
                                trackIssues = addOne(trackIssues,'noThirdChoice',oneName)
                    elif math.isnan(newPriority):
                        #print oneName.values.ravel()[1],oneName.values.ravel()[0], ': did not list a second league'
                        trackIssues = addOne(trackIssues,'noSecondChoice',oneName)
        del subset
    elif nRemaining == 0:
        print oneLeague, 'no more space in league'

League_5_Wednesday_Open no more space in league


In [192]:
#make a new dictionary for other leagues...concat with openLeagues when done
# do doubles and competitive league by hand...
otherLeagues = ['League_4_Men', 'League_7_Ladies' ,'League_Scrod']

leagueDict_other = {}
for name in otherLeagues:
    leagueDict_other[name] = pd.DataFrame()

In [193]:
#put league managers in for other
manager = 'League_4_Men'
r = regSorted.loc[(regSorted['lastname']=='Pijanowski') & (regSorted['firstname']=='Brian')]
leagueDict_other = addManager(leagueDict_other,r,manager)

manager = 'League_7_Ladies'
r = regSorted.loc[(regSorted['lastname']=='Galligan') & (regSorted['firstname']=='Natalie')]
leagueDict_other = addManager(leagueDict_other,r,manager)

manager = 'League_Scrod'
r = regSorted.loc[(regSorted['lastname']=='Pilotte') & (regSorted['firstname']=='Courtney Lee')]
leagueDict_other = addManager(leagueDict_other,r,manager)

In [194]:
#only have to go through once
for idx in range(0,nPeople):
    for oL in otherLeagues:    
        oneName = regSorted.loc[idx,['lastname', 'firstname']] 
        oneName = oneName.to_frame().transpose() #need this format to make nice output
        if regSorted.loc[idx,oL]=='Yes': #careful with trailing spaces
            leagueDict_other = addOne(leagueDict_other,oL,oneName)
            #now that I have added the person to a league, change their priority to NaN
            regTracking.loc[idx,oL] = 'yes'
            regSorted.loc[idx,oL] = np.nan

In [195]:
#combine the two dictionaries
leagueDict_all = leagueDict_open.copy()
leagueDict_all.update(leagueDict_other)

#combine the list of names
allLeagues = list(openLeagues)
allLeagues.extend(otherLeagues)

In [196]:
#export the results to a series of CSV files (CAREFUL...will delete existing files)
for fn in allLeagues:
    fName = '%s.csv' % fn
    #check if the file exists...delete if it does
    if os.path.isfile(fName):
        os.remove(fName)
        
    leagueDict_all[fn].to_csv(fName)

In [ ]:
#ran multiple times...order the leagues are randomized will change the output

In [197]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  23  people in this category
notInSecond :  3  people in this category
notInThird :  0  people in this category
noSecondChoice :  12  people in this category
noThirdChoice :  3  people in this category


['League_2_Monday_Open',
 'League_1_Sunday_Open',
 'League_6_Thursday_Open',
 'League_3_Monday_Night_Open',
 'League_5_Wednesday_Open',
 'League_Tuesday_Open']

In [179]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  23  people in this category
notInSecond :  0  people in this category
notInThird :  0  people in this category
noSecondChoice :  14  people in this category
noThirdChoice :  0  people in this category


['League_Tuesday_Open',
 'League_3_Monday_Night_Open',
 'League_6_Thursday_Open',
 'League_2_Monday_Open',
 'League_5_Wednesday_Open',
 'League_1_Sunday_Open']

In [161]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  16  people in this category
notInSecond :  0  people in this category
notInThird :  0  people in this category
noSecondChoice :  7  people in this category
noThirdChoice :  0  people in this category


['League_Tuesday_Open',
 'League_3_Monday_Night_Open',
 'League_2_Monday_Open',
 'League_1_Sunday_Open',
 'League_6_Thursday_Open',
 'League_5_Wednesday_Open']

In [143]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'
    
randLeagues

notInFirst :  29  people in this category
notInSecond :  1  people in this category
notInThird :  1  people in this category
noSecondChoice :  13  people in this category
noThirdChoice :  0  people in this category


['League_6_Thursday_Open',
 'League_1_Sunday_Open',
 'League_2_Monday_Open',
 'League_Tuesday_Open',
 'League_3_Monday_Night_Open',
 'League_5_Wednesday_Open']

In [125]:
for problem in issueNames:
    print problem, ': ', len(trackIssues[problem]), ' people in this category'

randLeagues

notInFirst :  23  people in this category
notInSecond :  0  people in this category
notInThird :  0  people in this category
noSecondChoice :  14  people in this category
noThirdChoice :  0  people in this category


['League_Tuesday_Open',
 'League_3_Monday_Night_Open',
 'League_6_Thursday_Open',
 'League_2_Monday_Open',
 'League_1_Sunday_Open',
 'League_5_Wednesday_Open']